In [1]:
import cplex
import cobra
from cobra import Model, Reaction, Metabolite
import numpy as np
import pickle

In [2]:
%%time
path="./"
modelName="EColi.xml"
name=path+modelName
model=cobra.io.read_sbml_model(name)

CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 124 ms


In [3]:
model

Name,e_coli_core
Memory address,7f958e96ee10
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


In [5]:
def decouple(model):
    toDecoupled=dict()
    toNonDecoupled=dict()
    inverse=dict()
    error = 10**-12
    decoupled=Model("Decoupled")
    metabolites=[]
    for m in model.metabolites:
        mm = Metabolite(m.id,m.name)
        mm.compartment=m.compartment
        metabolites.append(mm)
    decoupled.add_metabolites(metabolites)
    reactions=[]
    contBase=0
    contDecoupled=0
    reversibles=set()
    irreversibles=set()
    for r in model.reactions:
        toDecoupled[contBase]=[]
        lim_i=r.lower_bound
        lim_s=r.upper_bound
        
        if r.lower_bound<-1000:
            r.lower_bound=-1000
        if r.upper_bound>1000:
            r.upper_bound=1000
        model.objective=r
        model.objective_direction="min"
        minimum=model.slim_optimize()
        model.objective_direction="max"
        maximum=model.slim_optimize()
        r.lower_bound=lim_i
        r.upper_bound=lim_s
        right= (maximum>error) and (r.upper_bound)>0
        left=(minimum<-error) and (r.lower_bound<0)
        if abs(minimum)>error or abs(maximum)>error:
            if right and left:
                reaction1=Reaction(r.id,r.name)
                reaction1.lower_bound=0
                reaction1.upper_bound=r.upper_bound
                for key in r.metabolites.keys():
                    reaction1.add_metabolites({
                        key: r.metabolites[key],
                    })
                toNonDecoupled[contDecoupled]=contBase
                reversibles.add(contDecoupled)
                contDecoupled+=1
                
                reaction2=Reaction(r.id+"_rev",r.name+"_rev")
                reaction2.lower_bound=0
                reaction2.upper_bound=-r.lower_bound
                for key in r.metabolites.keys():
                    reaction2.add_metabolites({
                        key: -r.metabolites[key],
                    })
                reaction1.gene_reaction_rule=r.gene_reaction_rule
                reaction2.gene_reaction_rule=r.gene_reaction_rule
                reactions.append(reaction1)
                reactions.append(reaction2)
                toNonDecoupled[contDecoupled]=contBase
                toDecoupled[contBase]=[contDecoupled-1,contDecoupled]
                reversibles.add(contDecoupled)
                inverse[contDecoupled]=contDecoupled-1
                inverse[contDecoupled-1]=contDecoupled
                contDecoupled+=1
                
            if right and not left:
                r.lower_bound=0
                reactions.append(r)
                toNonDecoupled[contDecoupled]=contBase
                toDecoupled[contBase]=[contDecoupled]
                irreversibles.add(contDecoupled)
                contDecoupled+=1
        
            if left and not right:
                reaction=Reaction(r.id+"_i",r.name+"_i")
                reaction.upper_bound=-r.lower_bound
                reaction.lower_bound=0
                for key in r.metabolites.keys():
                    reaction.add_metabolites({
                        key: -r.metabolites[key],
                    })
                reactions.append(reaction)
                toNonDecoupled[contDecoupled]=contBase
                toDecoupled[contBase]=[contDecoupled]
                irreversibles.add(contDecoupled)
                contDecoupled+=1
        contBase+=1    
    decoupled.add_reactions(reactions)
    print("OK")
    return([decoupled,toDecoupled,toNonDecoupled,reversibles,irreversibles,inverse])

In [6]:
%%time
process=decouple(model)
model2=process[0]
toDecoupled=process[1]
toNonDecoupled=process[2]
reversibles=process[3]
irreversibles=process[4]
inverse=process[5]

OK
CPU times: user 195 ms, sys: 1.93 ms, total: 197 ms
Wall time: 197 ms


In [25]:
# Index of the reaction 'EX_glc__D_e_i'. This reaction is included in all the EFMs in 'e. coli core'
# except for a trivial 2-cycle. This reaction is taken as an additional constraint by using R_mainIndex = 1

mainIndex=28 

In [26]:
fileName=path+"toDecoupled.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump(toDecoupled, filehandle)

fileName=path+"toNonDecoupled.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump(toNonDecoupled, filehandle)
    
fileName=path+"reversibles.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump(reversibles, filehandle)

fileName=path+"irreversibles.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump(irreversibles, filehandle)
    
fileName=path+"inverse.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump(inverse, filehandle)
    
fileName=path+"indices.txt"
with open(fileName, 'wb') as filehandle:
    pickle.dump([mainIndex,n,m], filehandle)



In [10]:
A2=cobra.util.array.create_stoichiometric_matrix(model2, array_type="dense", dtype=None)
S=A2.copy()
AA=[]
for i in range(len(A2)):
    indices=[]
    values=[]
    for j in range(len(A2[i])):
        if not A2[i][j]==0:
            indices.append(j)
            values.append(A2[i][j])
    AA.append([indices,values])

In [11]:
def solving(n,m,f, SDok2, B, method=1,direction=0):
    num_decision_var = n
    num_constraints = m
    myProblem = cplex.Cplex() 
    myProblem.set_log_stream(None)
    myProblem.parameters.lpmethod.set(method)
    myProblem.set_error_stream(None)
    myProblem.set_warning_stream(None)
    myProblem.set_results_stream(None)
    myProblem.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
    for i in range(num_decision_var):
        myProblem.variables.set_lower_bounds(i, 0.0)
    
    for i in range(num_constraints):
        
        myProblem.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind= SDok2[i][0], val= SDok2[i][1])],
            rhs= [B[i]],
            names = ["c"+str(i)],
            senses = ["E"]
        )
    for i in range(num_decision_var):
        myProblem.objective.set_linear([(i, f[i])])
    if direction==0:
        myProblem.objective.set_sense(myProblem.objective.sense.minimize)
    else:
        myProblem.objective.set_sense(myProblem.objective.sense.maximize)
    myProblem.solve()
    status=myProblem.solution.get_status()
    if status==1:
        return [myProblem.solution.get_values(),myProblem.solution.basis.get_basis(),f]
    else:
        return [status] 

In [12]:
n=len(model2.reactions)
A2=[AA[0],AA[1]]
for i in range(2,len(AA)):
    A3=A2.copy()
    A3.append(AA[i])
    m=len(A3)
    f=np.zeros(n)
    B=np.zeros(m)
    B[m-1]=1
    sol=solving(n,m,f, A3, B)
    redundant=True
    if len(sol)>1 or not sol[0]==3:
        redundant=False
    if not redundant:
        A2.append(AA[i])

print(len(A2),np.linalg.matrix_rank(S))

fileName=path+"A2.txt"

with open(fileName, 'wb') as filehandle:
    pickle.dump(A2, filehandle)


63 63
